## Set up notebook

In [185]:
MY_API_KEY='7ca7d761c1c9d2b36a6b6d0857c78344f9b7f8b0'

In [186]:
import pandas as pd
import censusdata
import numpy as np
import matplotlib.pyplot as plt

In [187]:
def add_FIP(df):
    county_fip=[]
    tract_fip=[]
    block_fip=[]
    
    for county in df.index:
        county_=county.geo[0][1]+county.geo[1][1]
        tract_=county.geo[0][1]+county.geo[1][1]+county.geo[2][1]
        try:
            block_=county.geo[0][1]+county.geo[1][1]+county.geo[2][1]+county.geo[3][1]
            block_fip.append(block_)
            is_block=True
        except:
            is_block=False
        
        
        county_fip.append(county_)
        tract_fip.append(tract_)
        
        
        #place_name.append(county.name)
    df['County'] =county_fip
    df['Tract'] =tract_fip
    if is_block: 
        df['block_FIP'] =block_fip
    #df['place']=place_name
    return df

In [188]:
year=2019 #2019 
county_list=['005', '047', '081',  '061'] #bx, bk, manhattan, queens

# Means of Transportation to Work

B08301

In [189]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B08301'))

commute_variables={'B08301_001E':'Tot_commuters','B08301_002E':'Drive','B08301_010E':'Transit_all','B08301_011E':'Bus','B08301_012E':'Subway'}


Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08301_001E  | MEANS OF TRANSPORTATION TO WOR | !! Estimate Total:                                       | int  
B08301_002E  | MEANS OF TRANSPORTATION TO WOR | !! !! Estimate Total: Car, truck, or van:                | int  
B08301_003E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! Estimate Total: Car, truck, or van: Drove alone | int  
B08301_004E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! Estimate Total: Car, truck, or van: Carpooled:  | int  
B08301_005E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! !! Estimate Total: Car, truck, or van: Carpoole | int  
B08301_006E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! !! Estimate Total: Car, truck, or van: Carpoole | int  
B08301_007E  | MEANS OF TRANSPORTATION TO WOR | !! !! !! !! Estimate Total: Car, truck, or va

In [190]:
def make_commute_mode_df(county_code):
    commute_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(commute_variables.keys()),key=MY_API_KEY)
    commute_data.columns=commute_data.columns.map(commute_variables)
    
    commute_data=add_FIP(commute_data)
    return commute_data

## Count by job type for workforce 16 y/o and above

https://censusreporter.org/tables/C24050/

Pulling the 5 high groups:
- Management, business, science, and arts occupations: white collar type 1
- Service occupations: blue collar type 1
- Sales and office occupations: white collar type 2
- Natural resources, construction, and maintenance occupations: blue collar type 2
- Production, transportation, and material moving occupations: blue collar type 3

In [191]:
censusdata.printtable(censusdata.censustable('acs5', year, 'C24050'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
C24050_001E  | INDUSTRY BY OCCUPATION FOR THE | !! Estimate Total:                                       | int  
C24050_002E  | INDUSTRY BY OCCUPATION FOR THE | !! !! Estimate Total: Agriculture, forestry, fishing and | int  
C24050_003E  | INDUSTRY BY OCCUPATION FOR THE | !! !! Estimate Total: Construction                       | int  
C24050_004E  | INDUSTRY BY OCCUPATION FOR THE | !! !! Estimate Total: Manufacturing                      | int  
C24050_005E  | INDUSTRY BY OCCUPATION FOR THE | !! !! Estimate Total: Wholesale trade                    | int  
C24050_006E  | INDUSTRY BY OCCUPATION FOR THE | !! !! Estimate Total: Retail trade                       | int  
C24050_007E  | INDUSTRY BY OCCUPATION FOR THE | !! !! Estimate Total: Transportation and ware

In [192]:
#Management, business, science, and arts occupations: white collar type 1 (J_W_1)
#Service occupations: blue collar type 1 (J_B_1)
#Sales and office occupations: white collar type 2 (J_W_2)
#Natural resources, construction, and maintenance occupations: blue collar type 2 (J_B_2)
#Production, transportation, and material moving occupations: blue collar type 3 (J_B_3)
job_variables={'C24050_001E':'J_Tot','C24050_015E':'J_W_1','C24050_029E':'J_B_1','C24050_043E':'J_W_2',
               'C24050_057E':"J_B_2", 'C24050_071E':'J_B_3'}

In [193]:
def make_job_df(county_code):
    job_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(job_variables.keys()),key=MY_API_KEY)
    job_data.columns=job_data.columns.map(job_variables)
    
    job_data=add_FIP(job_data)
    return job_data

## Median HH income

In [194]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B19013'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | MEDIAN HOUSEHOLD INCOME IN THE | !! Estimate Median household income in the past 12 month | int  
-------------------------------------------------------------------------------------------------------------------


In [195]:
m_inc_variables={'B19013_001E':'Median_Inc'}

In [196]:
def make_m_inc_df(county_code):
    m_inc_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(m_inc_variables.keys()),key=MY_API_KEY)
    m_inc_data.columns=m_inc_data.columns.map(m_inc_variables)
    
    m_inc_data=add_FIP(m_inc_data)
    return m_inc_data

## Aggregated HH income

In [197]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B19025'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19025_001E  | AGGREGATE HOUSEHOLD INCOME IN  | !! Estimate Aggregate household income in the past 12 mo | int  
-------------------------------------------------------------------------------------------------------------------


In [198]:
a_inc_variables={'B19025_001E':'Agg_Inc'}

In [199]:
def make_a_inc_df(county_code):
    a_inc_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(a_inc_variables.keys()),key=MY_API_KEY)
    a_inc_data.columns=a_inc_data.columns.map(a_inc_variables)
    
    a_inc_data=add_FIP(a_inc_data)
    return a_inc_data

## Median age for workers 16 to 64 years

In [200]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B23013'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B23013_001E  | MEDIAN AGE BY SEX FOR WORKERS  | !! !! Estimate Median age-- Total:                       | float
B23013_002E  | MEDIAN AGE BY SEX FOR WORKERS  | !! !! !! Estimate Median age-- Total: Male               | float
B23013_003E  | MEDIAN AGE BY SEX FOR WORKERS  | !! !! !! Estimate Median age-- Total: Female             | float
-------------------------------------------------------------------------------------------------------------------


In [201]:
m_age_wor_variables = {'B23013_001E':'M_Age_Wor'}

In [202]:
def make_m_age_wor_df(county_code):
    m_age_wor_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(m_age_wor_variables.keys()),key=MY_API_KEY)
    m_age_wor_data.columns=m_age_wor_data.columns.map(m_age_wor_variables)
    
    m_age_wor_data=add_FIP(m_age_wor_data)
    return m_age_wor_data

## Medain age for all pop

In [203]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B01002'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B01002_001E  | MEDIAN AGE BY SEX              | !! !! Estimate Median age -- Total:                      | float
B01002_002E  | MEDIAN AGE BY SEX              | !! !! Estimate Median age -- Male                        | float
B01002_003E  | MEDIAN AGE BY SEX              | !! !! Estimate Median age -- Female                      | float
-------------------------------------------------------------------------------------------------------------------


In [204]:
m_age_all_variables = {'B01002_001E':'M_Age_All'}

In [205]:
def make_m_age_all_df(county_code):
    m_age_all_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(m_age_all_variables.keys()),key=MY_API_KEY)
    m_age_all_data.columns=m_age_all_data.columns.map(m_age_all_variables)
    
    m_age_all_data=add_FIP(m_age_all_data)
    return m_age_all_data

## Race and population

In [206]:
#Hsu designates five groups: Asian, Black, Hispanic, White, and no majority groups.
censusdata.printtable(censusdata.censustable('acs5', year, 'B03002'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B03002_001E  | HISPANIC OR LATINO ORIGIN BY R | !! Estimate Total:                                       | int  
B03002_002E  | HISPANIC OR LATINO ORIGIN BY R | !! !! Estimate Total: Not Hispanic or Latino:            | int  
B03002_003E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: White a | int  
B03002_004E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Black o | int  
B03002_005E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: America | int  
B03002_006E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Asian a | int  
B03002_007E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Lati

In [207]:
race_variables={'B03002_001E':'Race_Total','B03002_003E':'White','B03002_004E':'Black','B03002_012E':'Hispanic_Latino',
               'B03002_006E':"Asian", 'B03002_009E':'Mixed'}
#total-white-black-hispanic-asian-mixed=minor races

In [208]:
def make_race_df(county_code):
    race_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(race_variables.keys()),key=MY_API_KEY)
    race_data.columns=race_data.columns.map(race_variables)
    
    race_data=add_FIP(race_data)
    return race_data

In [209]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B19013'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | MEDIAN HOUSEHOLD INCOME IN THE | !! Estimate Median household income in the past 12 month | int  
-------------------------------------------------------------------------------------------------------------------


In [210]:
def make_income_df(county_code):
    income_data=censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       ['B19013_001E'],key=MY_API_KEY)
    income_data.columns=['Median_Income']
    income_data=add_FIP(income_data)
    

    return income_data

## Education attainment

https://censusreporter.org/tables/B15001/

In [211]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B15001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B15001_001E  | SEX BY AGE BY EDUCATIONAL ATTA | !! Estimate Total:                                       | int  
B15001_002E  | SEX BY AGE BY EDUCATIONAL ATTA | !! !! Estimate Total: Male:                              | int  
B15001_003E  | SEX BY AGE BY EDUCATIONAL ATTA | !! !! !! Estimate Total: Male: 18 to 24 years:           | int  
B15001_004E  | SEX BY AGE BY EDUCATIONAL ATTA | !! !! !! !! Estimate Total: Male: 18 to 24 years: Less t | int  
B15001_005E  | SEX BY AGE BY EDUCATIONAL ATTA | !! !! !! !! Estimate Total: Male: 18 to 24 years: 9th to | int  
B15001_006E  | SEX BY AGE BY EDUCATIONAL ATTA | !! !! !! !! Estimate Total: Male: 18 to 24 years: High s | int  
B15001_007E  | SEX BY AGE BY EDUCATIONAL ATTA | !! !! !! !! Estimate Total: Male: 18 to 24 ye

In [212]:
#high school and above
#bachelor's and above
#edu_tot as denominator
edu_variables = {'B15001_001E':'Edu_Tot', 'B15001_006E':'18m_hs', 'B15001_007E':'18m_some_c',
                 'B15001_008E':'18m_ass', 'B15001_009E':'18m_bac', 'B15001_010E':'18m_grad',
                 'B15001_014E':'25m_hs', 'B15001_015E':'25m_some_c', 'B15001_016E':'25m_ass',
                 'B15001_017E':'25m_bac', 'B15001_018E':'25m_grad',
                 'B15001_022E':'35m_hs', 'B15001_023E':'35m_some_c', 'B15001_024E':'35m_ass',
                 'B15001_025E':'35m_bac', 'B15001_026E':'35m_grad',
                 'B15001_030E':'45m_hs', 'B15001_031E':'45m_some_c', 'B15001_032E':'45m_ass',
                 'B15001_033E':'45m_bac', 'B15001_034E':'45m_grad',
                 'B15001_038E':'65m_hs', 'B15001_039E':'65m_some_c', 'B15001_040E':'65m_ass',
                 'B15001_041E':'65m_bac', 'B15001_042E':'65m_grad',
                 'B15001_047E':'18f_hs', 'B15001_048E':'18f_some_c', 'B15001_049E':'18f_ass', 
                 'B15001_050E':'18f_bac', 'B15001_051E':'18f_grad',
                 'B15001_055E':'25f_hs', 'B15001_056E':'25f_some_c', 'B15001_057E':'25f_ass',
                 'B15001_058E':'25f_bac', 'B15001_059E':'25f_grad',
                 'B15001_063E':'35f_hs', 'B15001_064E':'35f_some_c', 'B15001_065E':'35f_ass',
                 'B15001_066E':'35f_bac', 'B15001_067E':'35f_grad',
                 'B15001_071E':'45f_hs', 'B15001_072E':'45f_some_c', 'B15001_073E':'45f_ass',
                 'B15001_074E':'45f_bac', 'B15001_075E':'45f_grad',
                 'B15001_079E':'65f_hs', 'B15001_080E':'65f_some_c', 'B15001_081E':'65f_ass',
                 'B15001_082E':'65f_bac', 'B15001_083E':'65f_grad'}

In [213]:
def make_edu_df(county_code):
    edu_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(edu_variables.keys()), key=MY_API_KEY)
    edu_data.columns=edu_data.columns.map(edu_variables)
    
    edu_data=add_FIP(edu_data)
    return edu_data

## Prep for individual dataframes

In [214]:
job = []
m_inc = []
a_inc = []
m_age_wor = []
m_age_all = []
race = []
edu = []
commute=[]

for county in county_list:
    job.append(make_job_df(county))
    m_inc.append(make_m_inc_df(county))
    a_inc.append(make_a_inc_df(county))
    m_age_wor.append(make_m_age_wor_df(county))
    m_age_all.append(make_m_age_all_df(county))
    race.append(make_race_df(county))
    edu.append(make_edu_df(county))
    commute.append(make_commute_mode_df(county))

## Dataframe for Commute

In [215]:
commute_col = list(commute_variables.values())+['County', 'Tract']

commute_d = pd.DataFrame()
commute_df = pd.concat([commute_d, (pd.DataFrame(data=commute[0], columns=commute_col)),
                     (pd.DataFrame(data=commute[1], columns=commute_col)),
                     (pd.DataFrame(data=commute[2], columns=commute_col)),
                     (pd.DataFrame(data=commute[3], columns=commute_col))]).reset_index(drop=True)
modes=['Drive', 'Transit_all', 'Bus', 'Subway']
for i in range(len(modes)):
      commute_df['Perc_{}'.format(modes[i])]=(commute_df['{}'.format(modes[i])]/commute_df['Tot_commuters'])*100
commute_df.head(2)

,Tot_commuters,Drive,Transit_all,Bus,Subway,County,Tract,Perc_Drive,Perc_Transit_all,Perc_Bus,Perc_Subway
0,1929,341,1338,215,1123,36005,36005020000,17.677553,69.362364,11.145671,58.216693
1,2333,391,1527,638,889,36005,36005020501,16.759537,65.452207,27.346764,38.105444


## Dataframe for job type

In [216]:
job_col = list(job_variables.values())+['County', 'Tract']

job_d = pd.DataFrame()
job_df = pd.concat([job_d, (pd.DataFrame(data=job[0], columns=job_col)),
                     (pd.DataFrame(data=job[1], columns=job_col)),
                     (pd.DataFrame(data=job[2], columns=job_col)),
                     (pd.DataFrame(data=job[3], columns=job_col))]).reset_index(drop=True)
job_df.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,County,Tract
0,1934,420,622,383,175,334,36005,36005020000
1,2333,562,737,663,225,146,36005,36005020501


## Dataframe for median hh income

In [217]:
m_inc_col = list(m_inc_variables.values())+['County', 'Tract']

m_inc_d = pd.DataFrame()
m_inc_df = pd.concat([m_inc_d, (pd.DataFrame(data=m_inc[0], columns=m_inc_col)),
                     (pd.DataFrame(data=m_inc[1], columns=m_inc_col)),
                     (pd.DataFrame(data=m_inc[2], columns=m_inc_col)),
                     (pd.DataFrame(data=m_inc[3], columns=m_inc_col))]).reset_index(drop=True)
m_inc_df.head(2)

,Median_Inc,County,Tract
0,40184,36005,36005020000
1,17601,36005,36005020501


## Dataframe for aggregated hh income

In [218]:
a_inc_col = list(a_inc_variables.values())+['County', 'Tract']

a_inc_d = pd.DataFrame()
a_inc_df = pd.concat([a_inc_d, (pd.DataFrame(data=a_inc[0], columns=a_inc_col)),
                     (pd.DataFrame(data=a_inc[1], columns=a_inc_col)),
                     (pd.DataFrame(data=a_inc[2], columns=a_inc_col)),
                     (pd.DataFrame(data=a_inc[3], columns=a_inc_col))]).reset_index(drop=True)
a_inc_df.head(2)

,Agg_Inc,County,Tract
0,78328200.0,36005,36005020000
1,101308000.0,36005,36005020501


## Dataframe for median age for workers 16-64

In [219]:
m_age_wor_col = list(m_age_wor_variables.values())+['County', 'Tract']

m_age_wor_d = pd.DataFrame()
m_age_wor_df = pd.concat([m_age_wor_d, (pd.DataFrame(data=m_age_wor[0], columns=m_age_wor_col)),
                     (pd.DataFrame(data=m_age_wor[1], columns=m_age_wor_col)),
                     (pd.DataFrame(data=m_age_wor[2], columns=m_age_wor_col)),
                     (pd.DataFrame(data=m_age_wor[3], columns=m_age_wor_col))]).reset_index(drop=True)
m_age_wor_df.head(2)

,M_Age_Wor,County,Tract
0,42.0,36005,36005020000
1,37.1,36005,36005020501


## Dataframe for median age for all population

In [220]:
m_age_all_col = list(m_age_all_variables.values())+['County', 'Tract']

m_age_all_d = pd.DataFrame()
m_age_all_df = pd.concat([m_age_all_d, (pd.DataFrame(data=m_age_all[0], columns=m_age_all_col)),
                     (pd.DataFrame(data=m_age_all[1], columns=m_age_all_col)),
                     (pd.DataFrame(data=m_age_all[2], columns=m_age_all_col)),
                     (pd.DataFrame(data=m_age_all[3], columns=m_age_all_col))]).reset_index(drop=True)
m_age_all_df.head(2)

,M_Age_All,County,Tract
0,32.7,36005,36005020000
1,34.3,36005,36005020501


## Dataframe for race

The race categories here included are: 

*   White
*   Black
*   Hispanic or Latino
*   Asian
*   Other races
*   Mixed



In [221]:
race_col = ['Race_Total','White','Black',
            'Hispanic_Latino',"Asian",'Mixed','County', 'Tract']

race_d = pd.DataFrame()
race_df = pd.concat([race_d, (pd.DataFrame(data=race[0], columns=race_col)),
                     (pd.DataFrame(data=race[1], columns=race_col)),
                     (pd.DataFrame(data=race[2], columns=race_col)),
                     (pd.DataFrame(data=race[3], columns=race_col))]).reset_index(drop=True)

#total-white-black-hispanic-asian-mixed=minor races
race_df['Other'] = race_df['Race_Total']-race_df.iloc[:,1:-2].sum(axis=1)

#% of each race
races = ['White', 'Black',
        'Hispanic_Latino',"Asian",
        'Other', 'Mixed']
for i in range(len(races)):
      race_df['%{}'.format(races[i])]=(race_df['{}'.format(races[i])]/race_df['Race_Total'])*100

race_df = race_df.iloc[:, [0,6,7,9,10,11,12,13,14]]

race_df.head(2)

,Race_Total,County,Tract,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed
0,4711,36005,36005020000,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707
1,6976,36005,36005020501,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032


## Dataframe for education attainment

The data here is for people above 18 and hold a high school diploma or above. We will look at two types of education attainment: $\frac{Bachelor's\: or\: higher}{Population\: 18 \:and \:above}$, and $\frac{High\: School \: or\: higher}{Population\: 18 \:and \:above}$

In [222]:
edu_col = ['Edu_Tot', 
'18m_hs', '18m_some_c', '18m_ass', '18m_bac', '18m_grad',
'25m_hs', '25m_some_c', '25m_ass', '25m_bac', '25m_grad',
'35m_hs', '35m_some_c', '35m_ass', '35m_bac', '35m_grad',
'45m_hs', '45m_some_c', '45m_ass', '45m_bac', '45m_grad',
'65m_hs', '65m_some_c', '65m_ass', '65m_bac', '65m_grad',
'18f_hs', '18f_some_c', '18f_ass', '18f_bac', '18f_grad',
'25f_hs', '25f_some_c', '25f_ass', '25f_bac', '25f_grad',
'35f_hs', '35f_some_c', '35f_ass', '35f_bac', '35f_grad',
'45f_hs', '45f_some_c', '45f_ass', '45f_bac', '45f_grad',
'65f_hs', '65f_some_c', '65f_ass', '65f_bac', '65f_grad',
'County', 'Tract']

edu_d = pd.DataFrame()
edu_df = pd.concat([edu_d, (pd.DataFrame(data=edu[0], columns=edu_col)),
                     (pd.DataFrame(data=edu[1], columns=edu_col)),
                     (pd.DataFrame(data=edu[2], columns=edu_col)),
                     (pd.DataFrame(data=edu[3], columns=edu_col))]).reset_index(drop=True)

#% of adults with high school degree or above
edu_df['%Edu_hs'] = (((edu_df.iloc[:, 1:-2]).sum(axis=1))/edu_df['Edu_Tot'])*100
#of adults with bachelors or above
edu_df['%Edu_bac'] = (edu_df.iloc[:, [4,5,9,10,14,15,19,20,24,25,
                                      29,30,34,35,39,40,44,45,49,50]].sum(axis=1)/edu_df['Edu_Tot'])*100
edu_df = edu_df.iloc[:,[0, 51,52,53,54]]

edu_df.head(2)

,Edu_Tot,County,Tract,%Edu_hs,%Edu_bac
0,3310,36005,36005020000,71.873112,23.685801
1,5313,36005,36005020501,71.052136,9.655562


## Combine the four dataframes

In [223]:
#job_df
#m_inc_df
#a_inc_df
#m_age_wor_df
#m_age_all_df
#race_df
#edu_df

In [224]:
merge1 = job_df.merge(m_inc_df, how='inner', on='Tract')
merge1 = merge1.iloc[:, :-1]
merge1.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,County_x,Tract,Median_Inc
0,1934,420,622,383,175,334,36005,36005020000,40184
1,2333,562,737,663,225,146,36005,36005020501,17601


In [225]:
merge2 = merge1.merge(a_inc_df,how='inner',on='Tract')
merge2 = merge2.drop('County_x', axis=1)
merge2.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,Tract,Median_Inc,Agg_Inc,County
0,1934,420,622,383,175,334,36005020000,40184,78328200.0,36005
1,2333,562,737,663,225,146,36005020501,17601,101308000.0,36005


In [226]:
merge3 = merge2.merge(m_age_wor_df,how='inner',on='Tract')
merge3 = merge3.drop('County_y', axis=1)
merge3.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,Tract,Median_Inc,Agg_Inc,County_x,M_Age_Wor
0,1934,420,622,383,175,334,36005020000,40184,78328200.0,36005,42.0
1,2333,562,737,663,225,146,36005020501,17601,101308000.0,36005,37.1


In [227]:
merge4 = merge3.merge(m_age_all_df,how='inner',on='Tract')
merge4 = merge4.drop('County_x', axis=1)
merge4.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,Tract,Median_Inc,Agg_Inc,M_Age_Wor,M_Age_All,County
0,1934,420,622,383,175,334,36005020000,40184,78328200.0,42.0,32.7,36005
1,2333,562,737,663,225,146,36005020501,17601,101308000.0,37.1,34.3,36005


In [228]:
merge5 = merge4.merge(race_df,how='inner',on='Tract')
merge5 = merge5.drop('County_y', axis=1)
merge5.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,Tract,Median_Inc,Agg_Inc,M_Age_Wor,M_Age_All,County_x,Race_Total,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed
0,1934,420,622,383,175,334,36005020000,40184,78328200.0,42.0,32.7,36005,4711,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707
1,2333,562,737,663,225,146,36005020501,17601,101308000.0,37.1,34.3,36005,6976,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032


In [229]:
merge6 = merge5.merge(edu_df,how='inner',on='Tract')
merge6 = merge6.drop('County_x', axis=1)
merge6.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,Tract,Median_Inc,Agg_Inc,M_Age_Wor,...,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,Edu_Tot,County,%Edu_hs,%Edu_bac
0,1934,420,622,383,175,334,36005020000,40184,78328200.0,42.0,...,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707,3310,36005,71.873112,23.685801
1,2333,562,737,663,225,146,36005020501,17601,101308000.0,37.1,...,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032,5313,36005,71.052136,9.655562


In [230]:
merge6.columns

Index(['J_Tot', 'J_W_1', 'J_B_1', 'J_W_2', 'J_B_2', 'J_B_3', 'Tract',
       'Median_Inc', 'Agg_Inc', 'M_Age_Wor', 'M_Age_All', 'Race_Total',
       '%White', '%Black', '%Hispanic_Latino', '%Asian', '%Other', '%Mixed',
       'Edu_Tot', 'County', '%Edu_hs', '%Edu_bac'],
      dtype='object')

In [231]:
merge7= merge6.merge(commute_df,how='inner',on='Tract')
merge7 = merge7.drop('County_x', axis=1)
merge7.head(2)

,J_Tot,J_W_1,J_B_1,J_W_2,J_B_2,J_B_3,Tract,Median_Inc,Agg_Inc,M_Age_Wor,...,Tot_commuters,Drive,Transit_all,Bus,Subway,County_y,Perc_Drive,Perc_Transit_all,Perc_Bus,Perc_Subway
0,1934,420,622,383,175,334,36005020000,40184,78328200.0,42.0,...,1929,341,1338,215,1123,36005,17.677553,69.362364,11.145671,58.216693
1,2333,562,737,663,225,146,36005020501,17601,101308000.0,37.1,...,2333,391,1527,638,889,36005,16.759537,65.452207,27.346764,38.105444


In [232]:
merge7.columns

Index(['J_Tot', 'J_W_1', 'J_B_1', 'J_W_2', 'J_B_2', 'J_B_3', 'Tract',
       'Median_Inc', 'Agg_Inc', 'M_Age_Wor', 'M_Age_All', 'Race_Total',
       '%White', '%Black', '%Hispanic_Latino', '%Asian', '%Other', '%Mixed',
       'Edu_Tot', '%Edu_hs', '%Edu_bac', 'Tot_commuters', 'Drive',
       'Transit_all', 'Bus', 'Subway', 'County_y', 'Perc_Drive',
       'Perc_Transit_all', 'Perc_Bus', 'Perc_Subway'],
      dtype='object')

In [233]:
df_combined = merge7[['Tract', 'Race_Total', 'Edu_Tot', 
                           '%White', '%Black',
                           '%Hispanic_Latino',"%Asian",
                           '%Other', '%Mixed', '%Edu_hs', '%Edu_bac', 
                           'J_Tot', 'J_W_1', 'J_B_1', 'J_W_2', 'J_B_2',
                            'J_B_3', 'Median_Inc', 'Agg_Inc', 'M_Age_Wor', 'M_Age_All','Transit_all', 'Perc_Drive', 'Perc_Transit_all', 
                            'Perc_Bus', 'Perc_Subway']]
df_combined = df_combined.rename({'Race_Total':'Pop_Tot', 'Edu_Tot':'Pop_18'}, axis=1)
df_combined

,Tract,Pop_Tot,Pop_18,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,%Edu_hs,...,J_B_3,Median_Inc,Agg_Inc,M_Age_Wor,M_Age_All,Transit_all,Perc_Drive,Perc_Transit_all,Perc_Bus,Perc_Subway
0,36005020000,4711,3310,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707,71.873112,...,334,40184,78328200.0,42.0,32.7,1338,17.677553,69.362364,11.145671,58.216693
1,36005020501,6976,5313,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032,71.052136,...,146,17601,101308000.0,37.1,34.3,1527,16.759537,65.452207,27.346764,38.105444
2,36005020502,2160,1737,1.666667,26.435185,71.481481,0.138889,0.000000,0.277778,64.824410,...,110,18919,40917200.0,33.6,39.3,555,24.831461,62.359551,26.292135,33.820225
3,36005020900,4287,3297,3.825519,37.602053,57.872638,0.256590,0.209937,0.233263,74.309979,...,209,31190,64260200.0,41.0,36.4,1178,19.557416,70.454545,13.098086,57.356459
4,36005021001,8930,7185,2.183651,50.470325,36.125420,8.801792,1.209406,1.209406,85.901183,...,491,54076,252806100.0,40.3,39.0,3059,22.057169,68.849876,15.890164,51.496736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,36061021800,6236,5023,20.525978,58.611289,18.136626,1.860167,0.545221,0.320718,81.206450,...,382,49482,246019500.0,36.1,34.5,2236,7.613424,69.484152,4.940957,63.859540
2053,36061028300,7949,6726,19.335766,5.535287,68.964650,4.981759,0.314505,0.868034,80.255724,...,363,56855,216280200.0,36.6,39.4,3056,9.824301,75.624845,9.180896,63.226924
2054,36061029100,12165,9582,6.321414,5.318537,82.836005,4.069051,0.139745,1.315249,70.632436,...,491,47054,251395300.0,35.1,33.7,3893,12.388918,67.834117,7.109252,60.724865
2055,36061029500,8182,7073,34.808115,2.994378,56.318748,3.312149,0.378880,2.187729,77.095999,...,339,73514,285031300.0,37.6,38.0,3382,10.556020,70.693980,7.211538,63.482441


In [234]:
df_combined.columns

Index(['Tract', 'Pop_Tot', 'Pop_18', '%White', '%Black', '%Hispanic_Latino',
       '%Asian', '%Other', '%Mixed', '%Edu_hs', '%Edu_bac', 'J_Tot', 'J_W_1',
       'J_B_1', 'J_W_2', 'J_B_2', 'J_B_3', 'Median_Inc', 'Agg_Inc',
       'M_Age_Wor', 'M_Age_All', 'Transit_all', 'Perc_Drive',
       'Perc_Transit_all', 'Perc_Bus', 'Perc_Subway'],
      dtype='object')

In [235]:
df_combined.to_csv('../Data/demog_tract_level.csv',index=False)

## Merge with 2010 shoreclipped tract

In [236]:
import geopandas as gpd

In [237]:
nyc_tract = gpd.read_file('../Data/2010_Census_Tracts/2010_Census_Tracts.shp').set_crs(4326, allow_override=True)
nyc_tract.shape

(2055, 16)

In [238]:
nyc_tract.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Use internal point in the census tract shapefile:
*Internal point—The Census Bureau calculates an internal point (latitude and longitude coordinates) for each geographic area. For many geographic areas, the internal point is the centroid, the geographic center of the entity. For some irregularly shaped areas (such as those shaped like a crescent), the centroid may be located outside the boundaries of the entity. In such instances, the internal point is identified as a point inside the entity boundaries nearest to the centroid and, if possible, a point that is on land area, not water.*

In [239]:
nyc_tract.head(2)

,ctlabel,boro_code,boro_name,ct2010,boro_ct201,cdeligibil,ntacode,ntaname,puma,shape_leng,shape_area,geoid,Center_Lat,Center_Lon,area_km2,geometry
0,98,1,Manhattan,009800,1009800,I,MN19,Turtle Bay-East Midtown,3808,5534.20030845,1906016.47074,36061009800,40.755788,-73.967775,0.177075,"POLYGON ((-73.96433 40.75638, -73.96479 40.755..."
1,102,1,Manhattan,010200,1010200,I,MN17,Midtown-Midtown South,3807,5687.80243891,1860992.68163,36061010200,40.759424,-73.974449,0.172893,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."


In [240]:
nyc_tract = nyc_tract.iloc[:, [11, 12, 13, 15]]
nyc_tract.head(5)

,geoid,Center_Lat,Center_Lon,geometry
0,36061009800,40.755788,-73.967775,"POLYGON ((-73.96433 40.75638, -73.96479 40.755..."
1,36061010200,40.759424,-73.974449,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."
2,36061010400,40.760779,-73.977673,"POLYGON ((-73.97446 40.76229, -73.97491 40.761..."
3,36061011300,40.754775,-73.987870,"POLYGON ((-73.98412 40.75484, -73.98460 40.754..."
4,36061013000,40.772736,-73.964734,"POLYGON ((-73.96148 40.77432, -73.96194 40.773..."


In [241]:
tract_df = pd.DataFrame(nyc_tract)
tract_df.head(5)

,geoid,Center_Lat,Center_Lon,geometry
0,36061009800,40.755788,-73.967775,"POLYGON ((-73.96433 40.75638, -73.96479 40.755..."
1,36061010200,40.759424,-73.974449,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."
2,36061010400,40.760779,-73.977673,"POLYGON ((-73.97446 40.76229, -73.97491 40.761..."
3,36061011300,40.754775,-73.987870,"POLYGON ((-73.98412 40.75484, -73.98460 40.754..."
4,36061013000,40.772736,-73.964734,"POLYGON ((-73.96148 40.77432, -73.96194 40.773..."


In [242]:
#merge with the census df from before
tract_merged = df_combined.merge(tract_df,how='right',left_on='Tract', right_on='geoid')\
                          .drop(columns=['geoid'])
tract_merged.head(2)

,Tract,Pop_Tot,Pop_18,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,%Edu_hs,...,M_Age_Wor,M_Age_All,Transit_all,Perc_Drive,Perc_Transit_all,Perc_Bus,Perc_Subway,Center_Lat,Center_Lon,geometry
0,36061009800,7302,6646,75.458778,2.095316,9.832922,10.832649,0.136949,1.643385,95.78694,...,36.1,41.6,1469,8.229098,32.236120,0.438885,30.480579,40.755788,-73.967775,"POLYGON ((-73.96433 40.75638, -73.96479 40.755..."
1,36061010200,122,102,54.918033,5.737705,11.475410,1.639344,22.950820,3.278689,100.00000,...,37.5,39.7,16,11.111111,25.396825,3.174603,22.222222,40.759424,-73.974449,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."


In [243]:
tract_merged.shape

(2055, 29)

In [244]:
#create geodataframe to save as shapefile
tract_merged_gdf = gpd.GeoDataFrame(tract_merged, geometry='geometry')
tract_merged_gdf.head(2)

,Tract,Pop_Tot,Pop_18,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,%Edu_hs,...,M_Age_Wor,M_Age_All,Transit_all,Perc_Drive,Perc_Transit_all,Perc_Bus,Perc_Subway,Center_Lat,Center_Lon,geometry
0,36061009800,7302,6646,75.458778,2.095316,9.832922,10.832649,0.136949,1.643385,95.78694,...,36.1,41.6,1469,8.229098,32.236120,0.438885,30.480579,40.755788,-73.967775,"POLYGON ((-73.96433 40.75638, -73.96479 40.755..."
1,36061010200,122,102,54.918033,5.737705,11.475410,1.639344,22.950820,3.278689,100.00000,...,37.5,39.7,16,11.111111,25.396825,3.174603,22.222222,40.759424,-73.974449,"POLYGON ((-73.97124 40.76094, -73.97170 40.760..."


In [245]:
tract_merged_gdf.to_file('../Data/tract_with_info_v4.shp')  

<ipython-input-245-f43b6f2bf16a>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  tract_merged_gdf.to_file('../Data/tract_with_info_v4.shp')


In [246]:
# tract_points = tract_merged.iloc[:,[0,12,13]]
# tract_points.to_csv('tract_points.csv', index=False)